In [ ]:
from transformers import LlamaForCausalLM, LlamaConfig, AutoTokenizer, TrainingArguments, Trainer
from transformers.modeling_outputs import CausalLMOutputWithPast
import datasets
from datasets import load_dataset, Dataset
import torch

In [ ]:
context_length = 1024

In [ ]:
class CustomLlamaModel(LlamaForCausalLM):
    def __init__(self, config):
        super().__init__(config)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        position_ids=None,
        past_key_values=None,
        inputs_embeds=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        cache_position=None,
    ):
        # Ensure return_dict is True to work with Trainer properly
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            labels=labels,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=True,
            cache_position=cache_position,
        )

        # If labels are provided, calculate the loss
        # if labels is not None:
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(outputs.logits[:, :-1, :].contiguous().view(-1,
                            self.config.vocab_size), labels[:, 1:].contiguous().view(-1))

            return CausalLMOutputWithPast(
                loss=loss,
                logits=outputs.logits,
                past_key_values=outputs.past_key_values,
                hidden_states=outputs.hidden_states,
                attentions=outputs.attentions,
            )

        # If no labels, just return the original model output
        return outputs

In [ ]:
# Ensure your GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Configuration for a hypothetical 1B parameter model
config_1B = LlamaConfig(
    vocab_size=32000,
    hidden_size=1024,
    intermediate_size=4096,
    num_hidden_layers=24,
    num_attention_heads=16,
    max_position_embeddings=context_length,
    pad_token_id=2,
    torch_dtype="bfloat16"
)

In [ ]:
# Initialize the model with bfloat16 precision
model = CustomLlamaModel(config_1B)
# model = model.half()
model = model.to(device)  # Move model to GPU
model = model.train()  # Set model to training mode

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer.pad_token_id = tokenizer.eos_token_id  # Set pad token to end-of-sequence token

In [ ]:
# Prepare dataset (example using 'wikimedia/wikipedia', '20231101.en' subset)
dataset = load_dataset("D:/ai-stuff/datasets/wikipedia", "20231101.en")

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.05)

In [ ]:
exam_dataset = dataset["train"].select(range(2))
exam_dataset

In [ ]:
exam_dataset[0]

In [ ]:
tokenized_batches = tokenizer(
    exam_dataset["text"],
    padding="max_length",
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    stride=int(round(context_length / 4)),
    return_tensors="pt"
)

tokenized_batches

In [ ]:
for key in tokenized_batches:
    print(f"{key}: {tokenized_batches[key].shape}")

In [ ]:
# Decode and display the new tokenized text
decoded_text = []
for i in range(len(tokenized_batches["input_ids"])):
    decoded_text.append(tokenizer.decode(tokenized_batches["input_ids"][i]))

decoded_text

In [ ]:
small_train_dataset = dataset["train"].select(range(1000))
small_eval_dataset = dataset["test"].select(range(100))

In [ ]:
small_train_dataset

In [ ]:
small_eval_dataset

In [ ]:
dataset

In [ ]:
dataset

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    # Tokenize all texts and return overflow tokens as separate examples
    tokenized_batches = tokenizer(
        examples["text"],
        padding="max_length",  # or False if you prefer dynamic padding later on
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        stride=int(round(context_length / 4)),
        return_tensors="pt"
    )

    # Initialize lists to collect new examples
    overflow_input_ids = []
    overflow_labels = []

    for batch_index in range(len(tokenized_batches["input_ids"])):
        # If there are overflow tokens, they would be in 'overflowing_tokens' under 'encodings'
        num_overflows = len(
            tokenized_batches.encodings[batch_index].overflowing) if "overflowing_tokens" in tokenized_batches else 0

        # For the main input_ids batch and any overflow batches
        for overflow_index in range(num_overflows + 1):  # +1 to include the main batch itself
            if overflow_index == 0:  # The main batch
                input_ids = tokenized_batches["input_ids"][batch_index].tolist()
            else:  # Overflow batches
                input_ids = tokenized_batches.encodings[batch_index].overflowing[overflow_index - 1].ids

            labels = input_ids[1:] + [-100]  # Create labels shifted by one position
            overflow_input_ids.append(input_ids)
            overflow_labels.append(labels)

    # Prepare the final output as a dictionary
    output = {"input_ids": overflow_input_ids, "labels": overflow_labels}

    # Convert to Dataset format
    return datasets.Dataset.from_dict(output)

In [ ]:
tokenised_train_dataset = tokenize_function(small_train_dataset)
tokenised_train_dataset

In [ ]:
input_ids_lengths = [len(ids) for ids in tokenised_train_dataset['input_ids']]
min_length = min(input_ids_lengths)
max_length = max(input_ids_lengths)

print("Minimum length:", min_length)
print("Maximum length:", max_length)

In [ ]:
tokenized_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_eval = dataset["test"].map(tokenize_function, batched=True)

In [ ]:
# TrainingArguments setup
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # warmup_steps=5,
    learning_rate=5e-5,
    lr_scheduler_type="linear",
    logging_dir="./logs",
    logging_steps=1,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    # load_best_model_at_end=True,
    # metric_for_best_model="loss",
    gradient_accumulation_steps=8,
    bf16=True,  # Enable mixed-precision training
    bf16_full_eval=True,  # Enable mixed-precision evaluation
    optim="adamw_torch",  # Use PyTorch's AdamW optimizer
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

In [ ]:
# Start training
trainer.train()

In [ ]:
# Save the trained model
model.save_pretrained("./custom_llama_1B_model")

In [1]:
import json
import os
import time
import torch
from CustomLlamaModel import CustomLlamaModel
from datasets import load_dataset, Dataset
from transformers import LlamaConfig, AutoTokenizer, TrainingArguments, Trainer


# Model settings
hidden_layers = 8  # Number of transformer layers
hidden_size = 1024  # Size of the hidden states in the transformer layers
intermediate_size = 2048  # Size of the feed-forward network in the transformer layers
attention_heads = 32  # Number of attention heads
context_length = 2048  # Length of the input context
stride = 50  # Stride for splitting the input into multiple sequences

# Training settings
seed = 42
epochs = 20  # Number of training epochs
batch_size = 2  # Number of sequences to process in parallel
gradient_accumulation_steps = 10  # Number of update steps to accumulate before performing a backward pass
logging_steps = 1  # Log training loss every X steps
warmup_steps = 100 / gradient_accumulation_steps  # Number of warmup steps for the learning rate scheduler

run = "2"
output_dir = "./results/run-" + run
logging_dir = output_dir + "/logs"
final_dir = "./final"

learning_rate = 5e-5
lr_scheduler_type = "linear"
optim = "adamw_torch"  # Use PyTorch's AdamW optimizer

evaluation_strategy = "epoch"
eval_steps = 0.25
save_strategy = "epoch"
save_steps = 0.25

load_best_model_at_end = True
metric_for_best_model = "loss"

# Write the configuration to a JSON file
training_config = {
    "hidden_layers": hidden_layers,
    "hidden_size": hidden_size,
    "intermediate_size": intermediate_size,
    "attention_heads": attention_heads,
    "context_length": context_length,
    "stride": stride,
    "seed": seed,
    "epochs": epochs,
    "batch_size": batch_size,
    "gradient_accumulation_steps": gradient_accumulation_steps,
    "logging_steps": logging_steps,
    "warmup_steps": warmup_steps,
    "learning_rate": learning_rate,
    "lr_scheduler_type": lr_scheduler_type,
    "optim": optim,
    "evaluation_strategy": evaluation_strategy,
    "eval_steps": eval_steps,
    "save_strategy": save_strategy,
    "save_steps": save_steps,
    "load_best_model_at_end": load_best_model_at_end,
    "metric_for_best_model": metric_for_best_model,
    "start_time": time.strftime("%Y-%m-%d %H:%M:%S"),
}

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open(output_dir + "/training_config.json", "w") as f:
    json.dump(training_config, f, indent=4)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Configuration for a hypothetical 1B parameter model
config_1B = LlamaConfig(
    vocab_size=32000,
    hidden_size=hidden_size,
    intermediate_size=intermediate_size,
    num_hidden_layers=hidden_layers,
    num_attention_heads=attention_heads,
    max_position_embeddings=context_length,
    pad_token_id=2,
    torch_dtype="bfloat16"
)

# Initialize the model with bfloat16 precision
model = CustomLlamaModel(config_1B)
# model = model.half()  # Convert model parameters to bfloat16
model = model.to(device)  # Move model to GPU
model = model.train()  # Set model to training mode

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer.pad_token_id = tokenizer.eos_token_id  # Set pad token to end-of-sequence token

# Prepare dataset (example using 'wikimedia/wikipedia', '20231101.en' subset)
dataset = load_dataset("D:/ai-stuff/datasets/wikipedia", "20231101.en")

# Select the first 10000 examples
dataset = dataset["train"].select(range(10000))

# Merge the "title" and "text" columns into a single "text" column, separated by two newline characters
# dataset = dataset.map(lambda examples: {
#     "text": examples["title"] + "\n\n" + examples["text"]
# })

# Shuffling the dataset
dataset = dataset.shuffle(seed=seed)

# Split the dataset into training and evaluation sets 90:10
dataset = dataset.train_test_split(test_size=0.10)

Using device: cuda


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/39 [00:00<?, ?it/s]

In [4]:
# Tokenize the dataset
def tokenize_function(examples):
    # Tokenize all texts and return overflow tokens as separate examples
    tokenized_batches = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        stride=stride,
        return_tensors="pt"
    )

    # Shift the input ids to the left to create the labels so that the model predicts the next token.
    # The label for the last token is set to -100, so it's ignored by the loss function.
    tokenized_batches["labels"] = tokenized_batches.input_ids.clone()
    tokenized_batches["labels"][:, :-1] = tokenized_batches["labels"][:, 1:].clone()
    tokenized_batches["labels"][:, -1] = -100

    return tokenized_batches


def tokenize_function_2(examples):
    # Tokenize all texts and truncate them to the maximum length
    tokenized_batches = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=context_length,
        return_tensors="pt"
    )

    # Shift the input ids to the left to create the labels so that the model predicts the next token.
    # The label for the last token is set to -100, so it's ignored by the loss function.
    tokenized_batches["labels"] = tokenized_batches.input_ids.clone()
    tokenized_batches["labels"][:, :-1] = tokenized_batches["labels"][:, 1:].clone()
    tokenized_batches["labels"][:, -1] = -100

    return tokenized_batches

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 1000
    })
})

In [10]:
tokenized_train = tokenize_function(dataset["train"])
tokenized_eval = tokenize_function(dataset["test"])

In [9]:
train_tokens = dataset["train"].map(tokenize_function_2, batched=True)
test_tokens = dataset["test"].map(tokenize_function_2, batched=True)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
train_tokens

Dataset({
    features: ['id', 'url', 'title', 'text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 9000
})

In [13]:
tokenized_train

{'input_ids': tensor([[    1, 12009,   433,  ...,     2,     2,     2],
        [    1,   402,  1646,  ...,     2,     2,     2],
        [    1,   450,   306,  ...,  5925, 13457,   297],
        ...,
        [    1,   315,   804,  ...,   728,  1056, 28572],
        [    1, 13215, 15864,  ..., 29945,   259,   785],
        [    1, 29892,  3082,  ...,     2,     2,     2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'overflow_to_sample_mapping': tensor([   0,    1,    2,  ..., 8999, 8999, 8999]), 'labels': tensor([[12009,   433,  3851,  ...,     2,     2,  -100],
        [  402,  1646,  6626,  ...,     2,     2,  -100],
        [  450,   306, 29956,  ..., 13457,   297,  -100],
        ...,
        [  315,   804,   279,  ...,  1056, 28572,  -100],
        [13215, 15864, 13158,  ...,   259,   785,

In [14]:
tokenized_train_dataset = Dataset.from_dict(tokenized_train)

In [15]:
tokenized_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'overflow_to_sample_mapping', 'labels'],
    num_rows: 16535
})